In [219]:
import pandas as pd
import geopandas as gpd
import psycopg2 as psql
import numpy as np

In [90]:
def fetch_table(query, connect):
    '''
    Fetch table from database
    Input: str, str / Output: Dataframe
    '''
    cur = connect.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame(rows)
    cur.close()
    return df

In [122]:
db_name = 'SubwayDWT'
conn = psql.connect(host = 'localhost', port = 5432, user = 'postgres', password = 'user', database = db_name)
cur = conn.cursor()

In [123]:
query = 'SELECT * FROM subwayinfo_x'
raw_x = fetch_table(query, conn)

In [172]:
conn.close()

In [124]:
raw_x.rename(columns = {0: 'id', 3: 'name'}, inplace = True)

,id,1,2,name,4,5,6,7,8,9,10,11,12,13,14
0,6,01010000203B14000060DC46231D7D2D41F0384557DFA8...,5,가락시장,1,1,0,0,944586.7675563124951,805393.5386173954667,387790.658052818856,NaN,19987,76521,19
1,7,01010000203B140000D8ACFABCF0782D4108C58F81CFAB...,6,송파,0,1,0,0,1238646.417514794908,541334.2859829352654,9974.699793256976,NaN,31515,23419,16
2,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,62694.1253355969007,1777721.8252788334029,NaN,25099,81939,16
3,9,01010000203B140000D8ACFABCF0802D4170A301EC2BAA...,8,경찰병원,0,1,0,0,1232876.455101891585,141483.7233514069191,346489.697818391220,NaN,31749,34149,20
4,11,01010000203B14000028A913709A4F2D4140FAED7B66EA...,10,녹천,0,1,0,0,254261.54819983885256,13824.180336419040,3669.50489024873,NaN,12335,10066,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,297,01010000203B140000C070A6088DFA2C41D765E010C0A5...,296,신림,1,0,0,0,834894.65408275807474,483247.45814933421466,261036.23232184728148,NaN,41141,16700,31
298,298,01010000203B1400006A66001CD7FC2C41798A6C8EF3A2...,297,서원,0,0,0,0,999273.96541704114468,124335.08275455258367,14637.278241295510,NaN,36212,9058,21
299,299,01010000203B1400008BFA1E6E4EFD2C41532C4B1F43A0...,298,서울대벤처타운,0,0,0,0,565384.43078606770316,282429.8966665820014,34463.409654769173,NaN,32337,7071,23
300,301,01010000203B1400008B08B55F09032D4149C2E1F3BCA4...,300,봉천,0,0,0,1,927259.3994928983805,308414.2514051910808,72701.9598416250486,NaN,36447,13944,27


In [149]:
raw_y = pd.read_csv('../RawData/서울교통공사_역별 일별 시간대별 승하차인원 정보_20231031.csv', encoding = 'cp949')
raw_y9 = pd.read_csv('../RawData/서울교통공사_9호선2_3단계 역별일별시간대별승하차인원_20231031.csv', encoding = 'cp949')

raw_y9['06시이전'] = raw_y9['03-04시간대'] + raw_y9['04-05시간대'] + raw_y9['05-06시간대']
raw_y9['24시이후'] = raw_y9['00-01시간대'] + raw_y9['01-02시간대'] + raw_y9['02-03시간대']
raw_y9.drop(columns = ['03-04시간대','04-05시간대', '05-06시간대','00-01시간대', '01-02시간대', '02-03시간대'], inplace = True)

raw_y9['승하차구분'] = raw_y9['승하차구분'].str.replace('순승차', '승차')
raw_y9['승하차구분'] = raw_y9['승하차구분'].str.replace('순하차', '하차')

raw_y = pd.concat([raw_y, raw_y9])

In [159]:
new_rawy = raw_y.loc[:, ['수송일자', '역명', '승하차구분', '06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시이후']].groupby(['수송일자', '역명', '승하차구분']).sum().reset_index()

new_rawy = new_rawy.rename(columns = {'역명': 'name'})
# Join line info and transfer
def station_name_cut(x):
    x = x.split('(')[0]
    if x[-1] == '역':
        return x[:-1]
    else: return x
new_rawy['name'] = new_rawy['name'].apply(station_name_cut)

In [162]:
weekdays = ['2023-10-16', '2023-10-17','2023-10-18','2023-10-19','2023-10-20']
weekday_y = new_rawy[new_rawy['수송일자'].isin(weekdays)]
weekday_avg_y = weekday_y.groupby(['name', '승하차구분']).mean().reset_index()
weekday_seungcha_y = weekday_avg_y[weekday_avg_y['승하차구분'] == '승차']
weekday_hacha_y = weekday_avg_y[weekday_avg_y['승하차구분'] == '하차']

weekends = ['2023-10-21', '2023-10-22']
weekend_y = new_rawy[new_rawy['수송일자'].isin(weekends)]
weekend_avg_y = weekend_y.groupby(['name', '승하차구분']).mean().reset_index()
weekend_seungcha_y = weekend_avg_y[weekend_avg_y['승하차구분'] == '승차']
weekend_hacha_y = weekend_avg_y[weekend_avg_y['승하차구분'] == '하차']

C:\Users\Chulwoong Park\AppData\Local\Temp\ipykernel_27508\3892198869.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weekday_avg_y = weekday_y.groupby(['name', '승하차구분']).mean().reset_index()
C:\Users\Chulwoong Park\AppData\Local\Temp\ipykernel_27508\3892198869.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  weekend_avg_y = weekend_y.groupby(['name', '승하차구분']).mean().reset_index()


In [207]:
weekday_seungcha = raw_x.merge(weekday_seungcha_y, on = 'name')
weekday_hacha = raw_x.merge(weekday_hacha_y, on = 'name')
weekend_seungcha = raw_x.merge(weekend_seungcha_y, on = 'name')
weekend_hacha = raw_x.merge(weekend_hacha_y, on = 'name')

weekday_seungcha['weekend'] = 0
weekday_hacha['weekend'] = 0
weekend_seungcha['weekend'] = 1
weekend_hacha['weekend'] = 1

weekday_seungcha['hacha'] = 0
weekday_hacha['hacha'] = 1
weekend_seungcha['hacha'] = 0
weekend_hacha['hacha'] = 1

dataset = pd.concat([weekday_seungcha, weekday_hacha, weekend_seungcha, weekend_hacha])
dataset.to_csv('../Data/raw_dataset.csv')

In [210]:
dataset.reset_index(inplace = True)

In [211]:
# Get Max
dataset['max_ride'] = 0
dataset['min_ride'] = 0

for k, v in dataset.iterrows():
    dataset.iloc[k, -2] = v[['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']].max()
    dataset.iloc[k, -1] = v[['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']].min()
dataset.head(3)

,index,id,1,2,name,4,5,6,7,8,...,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후,weekend,hacha,max_ride,min_ride
0,0,6,01010000203B14000060DC46231D7D2D41F0384557DFA8...,5,가락시장,1,1,0,0,944586.7675563124951,...,977.4,795.6,730.8,603.8,250.6,63.4,0,0,2352.0,63.4
1,1,7,01010000203B140000D8ACFABCF0782D4108C58F81CFAB...,6,송파,0,1,0,0,1238646.417514794908,...,367.6,257.4,190.0,165.0,65.0,12.0,0,0,1710.6,12.0
2,2,8,01010000203B1400007836AB1EB67F2D41E0718A1E0FA6...,7,문정,0,1,0,0,595029.3087675537361,...,1828.4,1190.6,959.0,739.2,275.2,53.8,0,0,5700.4,53.8


In [221]:
# Get standardized
time_columns = ['06시이전', '06-07시간대',
       '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대',
       '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대',
       '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대',    '24시이후']

for k, v in dataset.iterrows():
    vols = v[time_columns].tolist()
    vmax, vmin = v['max_ride'], v['min_ride']
    normed_vols = [((i - vmin) / (vmax - vmin)) for i in vols]
    dataset.at[k, 'pattern'] = str(normed_vols)

In [224]:
#dataset_y = 
drop_cols = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, '승하차구분'] + time_columns
dataset_y = dataset.drop(columns = drop_cols)

In [228]:
dataset_y.to_csv('../Data/raw_y.csv')